####### Building an ETL Data Pipeline

- Step 1: Extract data from the csv file into a Pandas Dataframe
- Step 2: Transform the datat (i.e clean the data) - Deal with missing and duplicate data
- Step 3: Create a database
- Step 4: Load the clean data into the database



In [2]:
 # Import Libraries

import pandas as pd# For Data Extraction/transformation/wrangling/manipulation/analysis, etc
import psycopg2 # For connecting Python to Postgresql database
from sqlalchemy import create_engine # To efficiently manage and reuse the database connection
import os #To handle env variables

In [3]:
# Get the file path for the document/data
data = pd.read_csv(r"C:\Users\Josiah\Downloads\Softwork Technologies - Sheet1.csv")

# View the top five rows
data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,61


In [4]:
# View the bottom five rows
data.tail()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
23490,53478,Legal,region_2,Below Secondary,m,sourcing,1,24,3.0,1,0,61
23491,25600,Technology,region_25,Bachelor's,m,sourcing,1,31,3.0,7,0,74
23492,45409,HR,region_16,Bachelor's,f,sourcing,1,26,4.0,4,0,50
23493,1186,Procurement,region_31,Bachelor's,m,sourcing,3,27,NaN,1,0,70
23494,5973,Technology,region_17,Master's & above,m,other,3,40,5.0,5,0,89


### Step 2: Transform the datat (i.e clean the data) - Deal with missing and duplicate data

In [5]:
data.duplicated().sum()

5

In [6]:
# Remove duplicate records - Keep first or last
data.drop_duplicates(keep='first', inplace=True)

In [7]:
# Explore missing data
data.isnull().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
awards_won                 0
avg_training_score         0
dtype: int64

In [8]:
# Lets take a look at the missing data - EDUCATION COLUMN
data[data['education'].isnull()].head() # Top 5

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
49,3720,Sales & Marketing,region_2,NaN,m,sourcing,1,36,5.0,4,0,49
67,26977,Operations,region_2,NaN,m,sourcing,1,28,3.0,3,0,59
89,2347,Analytics,region_15,NaN,m,other,2,37,5.0,8,0,82
127,42719,Analytics,region_21,NaN,m,sourcing,1,27,NaN,1,0,80
138,2721,Analytics,region_22,NaN,f,sourcing,2,26,5.0,3,0,86


In [9]:
# Lets take a look at the missing data - PREVIOUS YEAR RATING COLUMN
data[data['previous_year_rating'].isnull()].head() # Top 5

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,0,77
21,5677,Technology,region_17,Bachelor's,m,sourcing,1,25,NaN,1,0,80
32,67672,Technology,region_17,Bachelor's,m,other,1,29,NaN,1,0,85
39,55325,Analytics,region_22,Bachelor's,m,other,1,25,NaN,1,0,88
47,44159,Analytics,region_22,Master's & above,m,other,1,31,NaN,1,0,84


In [10]:
# Deal with the missing data
data['education'].fillna('unknown', inplace=True) # For the education column
data['previous_year_rating'].fillna(0, inplace=True) # For the previous_year_rating column

In [11]:
data.isnull().sum()

employee_id             0
department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
awards_won              0
avg_training_score      0
dtype: int64

### Step 3: Create a database

Go to PG Admin and create database

In [15]:
# Database Credentials
username = os.environ['USER']
password = os.environ['PASS']
host = 'localhost'
port = 5433
db_name = 'postgres'


In [13]:
# Establish a connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db_name}')

In [14]:
#Load the database table - employee_table

data.to_sql('employee_table', engine, if_exists='replace', index=False)

# Close the connection
engine.dispose()